# Singapore Weather and Air Temperature Data Fetching (2015-2024)

This notebook fetches historical weather and air temperature data for Singapore (2015-2024) using the Open-Meteo API.
- **Weather Data**: Complete 2015-2024 dataset (temperature, humidity, wind speed)


In [14]:
# Install required packages
%pip install openmeteo-requests
%pip install requests-cache retry-requests numpy pandas


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [15]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 1.3521,
	"longitude": 103.8198,
	"start_date": "2015-01-01",
	"end_date": "2024-12-31",
	"hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m"],
	"timezone": "Asia/Singapore",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

# Create DataFrame
hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data sample:\n", hourly_dataframe.head())


Coordinates: 1.3708258867263794°N 103.8023681640625°E
Elevation: 46.0 m asl
Timezone: b'Asia/Singapore' b'GMT+8'
Timezone difference to GMT+0: 28800s

Hourly data sample:
                        date  temperature_2m  relative_humidity_2m  \
0 2014-12-31 16:00:00+00:00          24.257             90.525497   
1 2014-12-31 17:00:00+00:00          24.157             90.794037   
2 2014-12-31 18:00:00+00:00          24.007             91.615318   
3 2014-12-31 19:00:00+00:00          24.107             93.302505   
4 2014-12-31 20:00:00+00:00          24.157             92.741776   

   wind_speed_10m  
0        9.957108  
1       10.594036  
2       10.853866  
3       10.799999  
4       10.464798  


In [16]:
# Convert to daily data by taking average of hourly values for each day
hourly_dataframe['date'] = hourly_dataframe['date'].dt.date
daily_data = hourly_dataframe.groupby('date').mean()
daily_data = daily_data.reset_index()

# Add city name column
daily_data['city'] = 'Singapore'

# Display daily data
print("\nDaily data sample:\n", daily_data.head())



Daily data sample:
          date  temperature_2m  relative_humidity_2m  wind_speed_10m       city
0  2014-12-31       24.125750             91.975761       10.464737  Singapore
1  2015-01-01       24.621584             88.691841       11.648918  Singapore
2  2015-01-02       25.488251             78.113808       14.046573  Singapore
3  2015-01-03       25.882002             79.296227       13.742522  Singapore
4  2015-01-04       25.804916             80.059166       12.014789  Singapore


In [ ]:
# Save complete weather data for 2015-2024
import os

# Create directory if it doesn't exist
os.makedirs("/Users/sharin/Downloads/COS30049/Assignment/Assignment_2/COS30049-Computing-Technology-Innovation-Project-by-YSA/data/singapore/raw/weather", exist_ok=True)

# Add date_str column for filtering
daily_data['date_str'] = daily_data['date'].astype(str)

# Process each year from 2015 to 2024
for year in range(2015, 2025):
    # Filter data for the specific year
    year_data = daily_data[daily_data['date_str'].str.startswith(str(year))].copy()
    
    if len(year_data) > 0:
        # Format the data with all weather columns
        weather_data_year = pd.DataFrame({
            'date': pd.to_datetime(year_data['date']),
            'temperature_2m': year_data['temperature_2m'],
            'relative_humidity_2m': year_data['relative_humidity_2m'],
            'wind_speed_10m': year_data['wind_speed_10m']
        })
        
        # Save weather data for this year
        weather_file_path = f"/Users/sharin/Downloads/COS30049/Assignment/Assignment_2/COS30049-Computing-Technology-Innovation-Project-by-YSA/data/singapore/raw/weather/weather_{year}.csv"
        weather_data_year.to_csv(weather_file_path, index=False)
        
        print(f"Weather data ({year}) saved to: weather_{year}.csv - {len(weather_data_year)} records")

print(f"\nAll weather data (2015-2024) saved successfully!")


✅ Weather data (2015) saved to: weather_2015.csv - 365 records
✅ Weather data (2016) saved to: weather_2016.csv - 366 records
✅ Weather data (2017) saved to: weather_2017.csv - 365 records
✅ Weather data (2018) saved to: weather_2018.csv - 365 records
✅ Weather data (2019) saved to: weather_2019.csv - 365 records
✅ Weather data (2020) saved to: weather_2020.csv - 366 records
✅ Weather data (2021) saved to: weather_2021.csv - 365 records
✅ Weather data (2022) saved to: weather_2022.csv - 365 records
✅ Weather data (2023) saved to: weather_2023.csv - 365 records
✅ Weather data (2024) saved to: weather_2024.csv - 366 records

✅ All weather data (2015-2024) saved successfully!


In [ ]:
# Save air temperature data for all years 2015-2024
import os

# Create directory if it doesn't exist
os.makedirs("/Users/sharin/Downloads/COS30049/Assignment/Assignment_2/COS30049-Computing-Technology-Innovation-Project-by-YSA/data/singapore/raw/air_temperature", exist_ok=True)

# Process each year from 2015 to 2024
for year in range(2015, 2025):
    # Filter data for the specific year (date_str was already created in previous cell)
    year_data = daily_data[daily_data['date_str'].str.startswith(str(year))].copy()
    
    if len(year_data) > 0:
        # Format the data to match the existing air temperature format
        temp_data_year = pd.DataFrame({
            'timestamp': pd.to_datetime(year_data['date']),
            'reading_type': 'DBT 1M F',
            'station_name': 'Singapore_National',
            'reading_value': year_data['temperature_2m']
        })
        
        # Save temperature data for this year
        temp_file_path = f"/Users/sharin/Downloads/COS30049/Assignment/Assignment_2/COS30049-Computing-Technology-Innovation-Project-by-YSA/data/singapore/raw/air_temperature/airtemp_{year}.csv"
        temp_data_year.to_csv(temp_file_path, index=False)
        
        print(f"Air temperature data ({year}) saved to: airtemp_{year}.csv - {len(temp_data_year)} records")

print(f"\n All air temperature data (2015-2024) saved successfully!")


✅ Air temperature data (2015) saved to: airtemp_2015.csv - 365 records
✅ Air temperature data (2016) saved to: airtemp_2016.csv - 366 records
✅ Air temperature data (2017) saved to: airtemp_2017.csv - 365 records
✅ Air temperature data (2018) saved to: airtemp_2018.csv - 365 records
✅ Air temperature data (2019) saved to: airtemp_2019.csv - 365 records
✅ Air temperature data (2020) saved to: airtemp_2020.csv - 366 records
✅ Air temperature data (2021) saved to: airtemp_2021.csv - 365 records
✅ Air temperature data (2022) saved to: airtemp_2022.csv - 365 records
✅ Air temperature data (2023) saved to: airtemp_2023.csv - 365 records
✅ Air temperature data (2024) saved to: airtemp_2024.csv - 366 records

✅ All air temperature data (2015-2024) saved successfully!
